In [10]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
import pickle
import nltk

In [11]:
df=pd.read_csv("IMDB Dataset.csv")

In [12]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [13]:
def sentiment_to_numeric(sentiment):
    if sentiment.lower() == 'positive':
        return 1
    elif sentiment.lower() == 'negative':
        return 0
    else:
        raise ValueError("Invalid sentiment. Only 'positive' and 'negative' are allowed.")

In [14]:
def convert_sentiment_column_to_numeric(df, column_name):
    df[column_name] = df[column_name].apply(sentiment_to_numeric)

In [15]:
convert_sentiment_column_to_numeric(df, 'sentiment')

In [16]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [17]:
Vect=CountVectorizer()
docs = np.array(['This is the first document.',
       'This document is the second document.',
       'And this is the third one.',
       'Is this the first document?'])
bag=Vect.fit_transform(docs)

In [18]:
np.set_printoptions(precision=2)
tfidf=TfidfTransformer(use_idf=True,norm='l2',smooth_idf=True)
print(tfidf.fit_transform(bag).toarray())

[[0.   0.47 0.58 0.38 0.   0.   0.38 0.   0.38]
 [0.   0.69 0.   0.28 0.   0.54 0.28 0.   0.28]
 [0.51 0.   0.   0.27 0.51 0.   0.27 0.51 0.27]
 [0.   0.47 0.58 0.38 0.   0.   0.38 0.   0.38]]


In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mails\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [20]:
tfidf=TfidfVectorizer(
                      use_idf=True,
                      norm='l2',
                      smooth_idf=True)

In [21]:
y=df.sentiment.values
X=tfidf.fit_transform(df['review'].values.astype('U'))

In [22]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=1,test_size=0.5,shuffle=False)

In [23]:
clf=LogisticRegressionCV(cv=5,scoring='accuracy',
                        random_state=0,
                        n_jobs=-1,
                        verbose=3,
                        max_iter=300).fit(X_train,y_train)

saved_model=open('saved_model.sav', 'wb')
pickle.dump(clf,saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.6min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished


In [24]:
filename='saved_model.sav'
saved_clf=pickle.load(open(filename,'rb'))

saved_clf.score(X_test,y_test)

0.89716

In [25]:
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))

In [26]:
def analyze_sentiment(review):
        review_vectorized = tfidf.transform([review])
        prediction = saved_clf.predict(review_vectorized)
        if prediction[0]==1:
            return 'Positive'
        else:
            return 'Negative'

In [27]:
tfidf = TfidfVectorizer(use_idf=True, norm='l2', smooth_idf=True)

In [28]:
df=pd.read_csv("IMDB Dataset.csv")
X_train = tfidf.fit_transform(df['review'].values.astype('U'))

In [29]:
user_review = input("Enter your movie review: ")

Enter your movie review: good watching


In [30]:
result = analyze_sentiment(user_review)
print("Sentiment Analysis Result: ", result)

Sentiment Analysis Result:  Positive
